In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [31]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            print(mfccs.shape)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            print(chroma.shape)
            result=np.hstack((result, chroma))
        if mel:
                    mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
                    print(mel.shape)
                    result=np.hstack((result, mel))
    return result

In [32]:
f = extract_feature('C:\\Users\\nakul\\Downloads\\x.wav', mfcc=True, chroma=True, mel=True)

(40,)
(12,)
(128,)


<ipython-input-31-b77458668980>:18: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.01705933 -0.01791382
 -0.01858521] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


In [3]:
emotions = {
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

observed_emotions = [x for x in emotions.values()]

In [8]:
#DataFlair - Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("../ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=5322)

In [9]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

<ipython-input-2-aa76ece0c3b0>:16: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
<ipython-input-2-aa76ece0c3b0>:16: FutureWarning: Pass y=[ 3.0517578e-05  3.0517578e-05  3.0517578e-05 ...  0.0000000e+00
  0.0000000e+00 -3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
<ipython-input-2-aa76ece0c3b0>:16: FutureWarning: Pass y=[3.0517578e-05 3.0517578e-05 0.0000000e+00 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
<ipython-input-2-aa76ece0c3b0>:16: FutureWarning: Pass y=[0.0000000e+00 3.0517578e-05 3.0517578e-05 ... 

In [13]:
mu = x_train.mean(axis=0)
std = np.std(x_train, axis=0)

x_train = (x_train - mu) / std
x_test = (x_test - mu) / std

In [33]:
y_train

['happy',
 'surprised',
 'calm',
 'sad',
 'happy',
 'neutral',
 'happy',
 'angry',
 'angry',
 'calm',
 'surprised',
 'fearful',
 'calm',
 'disgust',
 'surprised',
 'surprised',
 'angry',
 'surprised',
 'fearful',
 'surprised',
 'surprised',
 'sad',
 'disgust',
 'happy',
 'surprised',
 'neutral',
 'surprised',
 'disgust',
 'disgust',
 'surprised',
 'fearful',
 'fearful',
 'sad',
 'happy',
 'neutral',
 'calm',
 'angry',
 'surprised',
 'sad',
 'angry',
 'angry',
 'calm',
 'fearful',
 'calm',
 'calm',
 'calm',
 'fearful',
 'fearful',
 'angry',
 'neutral',
 'neutral',
 'happy',
 'calm',
 'sad',
 'happy',
 'fearful',
 'happy',
 'neutral',
 'surprised',
 'calm',
 'surprised',
 'fearful',
 'fearful',
 'calm',
 'calm',
 'neutral',
 'angry',
 'neutral',
 'disgust',
 'happy',
 'fearful',
 'surprised',
 'angry',
 'fearful',
 'fearful',
 'surprised',
 'happy',
 'sad',
 'happy',
 'calm',
 'neutral',
 'calm',
 'angry',
 'surprised',
 'surprised',
 'fearful',
 'neutral',
 'calm',
 'surprised',
 'neutr

In [27]:
x_train.shape

(1080, 180)

In [14]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [15]:
#DataFlair - Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [16]:
#DataFlair - Predict for the test set
y_pred=model.predict(x_test)
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 60.56%


In [28]:
f = extract_feature('C:\\Users\\nakul\\Downloads\\x.wav', mfcc=True, chroma=True, mel=True)

<ipython-input-2-aa76ece0c3b0>:16: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.01705933 -0.01791382
 -0.01858521] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)


In [22]:
f = (f-mu)/std

In [26]:
model.predict_proba(f.reshape(1, -1))

array([[5.90499325e-063, 1.60656882e-096, 2.97594640e-122,
        3.19355221e-005, 1.26693062e-067, 1.67607119e-183,
        6.16619996e-185, 9.99968064e-001]])

In [35]:
y_test

['happy',
 'angry',
 'angry',
 'fearful',
 'angry',
 'disgust',
 'surprised',
 'disgust',
 'calm',
 'surprised',
 'angry',
 'sad',
 'angry',
 'sad',
 'sad',
 'calm',
 'surprised',
 'sad',
 'happy',
 'fearful',
 'fearful',
 'surprised',
 'calm',
 'calm',
 'disgust',
 'sad',
 'angry',
 'happy',
 'happy',
 'surprised',
 'fearful',
 'calm',
 'surprised',
 'surprised',
 'fearful',
 'surprised',
 'fearful',
 'calm',
 'disgust',
 'sad',
 'sad',
 'calm',
 'surprised',
 'surprised',
 'disgust',
 'calm',
 'sad',
 'sad',
 'angry',
 'fearful',
 'sad',
 'calm',
 'happy',
 'happy',
 'happy',
 'happy',
 'calm',
 'happy',
 'calm',
 'angry',
 'calm',
 'surprised',
 'calm',
 'happy',
 'disgust',
 'sad',
 'calm',
 'angry',
 'surprised',
 'fearful',
 'sad',
 'fearful',
 'happy',
 'angry',
 'disgust',
 'neutral',
 'sad',
 'calm',
 'disgust',
 'angry',
 'calm',
 'neutral',
 'sad',
 'disgust',
 'sad',
 'fearful',
 'angry',
 'disgust',
 'neutral',
 'sad',
 'angry',
 'surprised',
 'angry',
 'disgust',
 'happy'